In [3]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.autograd import Variable


In [7]:
# Prepare Data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

In [59]:
# LeNet Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = BinaryFunction()(x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [60]:
model = Net()
model.cuda()


Net (
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d (p=0.5)
  (fc1): Linear (320 -> 50)
  (fc2): Linear (50 -> 10)
)

In [61]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [62]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #if args.cuda:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
100. * batch_idx / len(train_loader), loss.data[0]))

In [63]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        #if args.cuda:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        #pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        pred = torch.max(output.data,1)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [68]:

train(1)
test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310760
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305841
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.316176
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.317196
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.303140
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.293335
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.309552
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303401
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.301362
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.308873
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.304788
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.292967
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.300804
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.302018
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.301513
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.299364
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.298456
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.303680
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.297683
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.307135
Train Epoch: 1 [12800/60000 (

In [3]:
from torch.autograd import Function

class BinaryFunction(Function):
    @staticmethod
    def forward(ctx, input):
        output=input.new(input.size())
        output[input>=0]=1
        output[input<0]=-1
        return output
    
    @staticmethod
    def backward(ctx, grad_output):
        grad_input=grad_output.clone()

        return grad_input

# aliases
binarise = BinaryFunction.apply


AttributeError: type object 'BinaryFunction' has no attribute 'apply'

In [49]:
input = Variable(torch.randn(20,20).double(), requires_grad=True)

In [50]:
output = BanaryFunction(input)

In [51]:
out=output.forward(input)

In [52]:
out.data



Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0    

In [1]:
from functions import *

AttributeError: type object 'BinarizeFunc' has no attribute 'apply'